# Taller 1. Vectorización

Sergio Herrera Quijano,
Oscar Bautista

In [ ]:
!pip install pandas requests
!pip install unidecode
!pip install nltk
!python -m spacy download es_core_news_sm
import pandas as pd
import numpy as np
import requests
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from io import StringIO
from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

### Descargar stopwords
nltk.download('stopwords')
nltk.download('punkt')




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 81.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Corrected URL to access the raw content
url = 'https://raw.githubusercontent.com/Izainea/nlp_ean/main/Datos/Datos%20Crudos/reviews_booking.csv'

# Fetching the raw content from the URL
response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad status codes
Corpus_DF = pd.read_csv(StringIO(response.text), sep=',', encoding='utf-8', on_bad_lines='skip')

Corpus_DF

<ipython-input-3-ac3a3652881b>:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Corpus_DF = pd.read_csv(StringIO(response.text), sep=',', encoding='utf-8', on_bad_lines='skip')


,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
1,1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10,"Ubicación, comodidades y limpieza",NaN,Sí
2,2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10,"El lugar esta muy bien ubicado, el apartamento...",NaN,Sí
3,3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10,Todo estuvo perfecto. La ubicación y la relaci...,NaN,Sí
4,4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10,"Es muy cómodo y acogedor, la atención es excel...",NaN,Sí
...,...,...,...,...,...,...,...,...,...,...,...,...
438208,438208,Estados Unidos,Apartamento Suite,1 noche,septiembre de 2024,En familia,22 de octubre de 2024,Fantástico,"9,0",NaN,NaN,No
438209,438209,Colombia,Apartamento de 1 dormitorio,1 noche,octubre de 2024,Persona que viaja sola,9 de octubre de 2024,Excepcional,10,NaN,NaN,No
438210,438210,México,Apartamento de 1 dormitorio,4 noches,agosto de 2024,Persona que viaja sola,6 de septiembre de 2024,Pésimo,"1,0",No me pareció nada,No obstante que dos días antes de mi llegada a...,Sí
438211,438211,Colombia,Apartamento de 1 dormitorio,2 noches,julio de 2024,En pareja,22 de julio de 2024,Excepcional,10,NaN,NaN,No


# Preprocesamiento:

In [ ]:
# Filtrar el DataFrame
Corpus_DF = Corpus_DF[
    Corpus_DF['Cosas Positivas'].notna() &
    (Corpus_DF['País'] == 'Colombia') &
    (pd.to_numeric(Corpus_DF['Calificación'], errors='coerce') >= 7)  # Convertir a numérico, manejar errores
]
Corpus_DF = Corpus_DF[
    Corpus_DF['Cosas Negativas'].notna() &
    (Corpus_DF['País'] == 'Colombia') &
    (pd.to_numeric(Corpus_DF['Calificación'], errors='coerce') >= 7)  # Convertir a numérico, manejar errores
]

# Función para eliminar repeticiones de vocales y consonantes
def remove_repeated_chars(text):
    # Eliminar repeticiones de vocales (> 1 vez)
    text = re.sub(r'([aeiouáéíóú])\1+', r'\1', text)
    # Eliminar repeticiones de consonantes (> 3 veces)
    text = re.sub(r'([^aeiouáéíóú\s\d])\1{3,}', r'\1', text)
    return text

# Limpiar y procesar la columna 'Cosas Positivas'
Corpus_DF['Cosas_Positivas_limpias'] = (Corpus_DF['Cosas Positivas']
    .str.lower()  # Convertir a minúsculas
    .str.replace(r'[^\w\s]', ' ', regex=True)  # Eliminar caracteres no alfanuméricos
    .str.strip()  # Eliminar espacios al inicio y al final
    .str.replace(r'\d+', '', regex=True)  # Eliminar números
    .apply(remove_repeated_chars)  # Aplicar la eliminación de repeticiones
)
Corpus_DF['Cosas_Negativas_limpias'] = (Corpus_DF['Cosas Negativas']
    .str.lower()  # Convertir a minúsculas
    .str.replace(r'[^\w\s]', ' ', regex=True)  # Eliminar caracteres no alfanuméricos
    .str.strip()  # Eliminar espacios al inicio y al final
    .str.replace(r'\d+', '', regex=True)  # Eliminar números
    .apply(remove_repeated_chars)  # Aplicar la eliminación de repeticiones
)
Corpus_DF

,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña,Cosas_Positivas_limpias,Cosas_Negativas_limpias
5,5,Colombia,NaN,1 noche,septiembre de 2023,Persona que viaja sola,5 de septiembre de 2023,Como en casa !,10,La atención y la tranquilidad tiene todo lo qu...,Nada,Sí,la atención y la tranquilidad tiene todo lo qu...,nada
6,6,Colombia,NaN,1 noche,julio de 2023,En pareja,2 de agosto de 2023,Excepcional,10,Las instalaciones,X,Sí,las instalaciones,x
7,7,Colombia,NaN,11 noches,julio de 2023,Persona que viaja sola,21 de julio de 2023,Excelente lugar!,10,"La limpieza, todo tal cual la foto, pero lo me...",Todo estuvo muy bien. Recomendado al 100%,Sí,la limpieza todo tal cual la foto pero lo me...,todo estuvo muy bien recomendado al
8,8,Colombia,NaN,1 noche,junio de 2023,En pareja,2 de julio de 2023,Acogedor,10,Apto cómodo y acogedor. Cerca a las zonas de i...,Todo perfecto.,Sí,apto cómodo y acogedor cerca a las zonas de i...,todo perfecto
49,49,Colombia,NaN,1 noche,mayo de 2024,En pareja,7 de mayo de 2024,Excepcional,10,La ubicación y las instalaciones,Que no ofrecen desayuno u otra comida dentro d...,Sí,la ubicación y las instalaciones,que no ofrecen desayuno u otra comida dentro d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438164,438164,Colombia,Apartamento de 1 dormitorio,1 noche,septiembre de 2024,Persona que viaja sola,7 de septiembre de 2024,Excelente apto,10,"La comodidad del apto, lo amplio.",Quizás hubo una pequeña confusión a la hora de...,Sí,la comodidad del apto lo amplio,quizás hubo una pequeña confusión a la hora de...
438166,438166,Colombia,Habitación Doble con baño compartido,1 noche,octubre de 2024,En pareja,26 de octubre de 2024,servicio y calidad,10,todo super bien gracias,todo muy bien gracias,Sí,todo super bien gracias,todo muy bien gracias
438176,438176,Colombia,Apartamento de 1 dormitorio,4 noches,agosto de 2024,En pareja,22 de agosto de 2024,Excepcional,10,"La anfitriona excelente muy atenta, pendiente ...",Hace falta mayor visibilizacion tanto en la no...,Sí,la anfitriona excelente muy atenta pendiente ...,hace falta mayor visibilizacion tanto en la no...
438177,438177,Colombia,Apartamento Superior de 2 dormitorios,1 noche,octubre de 2024,Persona que viaja sola,30 de octubre de 2024,Genial,10,La limpieza,Todo estuvo bien,Sí,la limpieza,todo estuvo bien


# Eliminación de Stopwords y Tokenización

In [ ]:
# Descargar recursos necesarios de NLTK
nltk.download('punkt_tab')
nltk.download('punkt')  # Tokenizador de palabras
nltk.download('stopwords')  # Palabras vacías (stopwords)

# Crear una lista combinada de palabras vacías en español, inglés y alemán
stop_words_spanish = set(stopwords.words('spanish'))
stop_words_english = set(stopwords.words('english'))
stop_words_german = set(stopwords.words('german'))
combined_stop_words = stop_words_spanish | stop_words_english | stop_words_german

# Función para eliminar tildes de una palabra
def remove_accents(word):
    return unidecode(word)

# Tokenizar, eliminar palabras vacías y quitar tildes
Corpus_DF['Tokens_Cosas_Positivas_limpias'] = Corpus_DF['Cosas_Positivas_limpias'].apply(
    lambda text: [
        remove_accents(word) for word in word_tokenize(text)
        if word.lower() not in combined_stop_words
    ]
)
Corpus_DF['Tokens_Cosas_Negativas_limpias'] = Corpus_DF['Cosas_Negativas_limpias'].apply(
    lambda text: [
        remove_accents(word) for word in word_tokenize(text)
        if word.lower() not in combined_stop_words
    ]
)
Corpus_DF

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña,Cosas_Positivas_limpias,Cosas_Negativas_limpias,Tokens_Cosas_Positivas_limpias,Tokens_Cosas_Negativas_limpias
5,5,Colombia,NaN,1 noche,septiembre de 2023,Persona que viaja sola,5 de septiembre de 2023,Como en casa !,10,La atención y la tranquilidad tiene todo lo qu...,Nada,Sí,la atención y la tranquilidad tiene todo lo qu...,nada,"[atencion, tranquilidad, necesita, ir, laborar]",[]
6,6,Colombia,NaN,1 noche,julio de 2023,En pareja,2 de agosto de 2023,Excepcional,10,Las instalaciones,X,Sí,las instalaciones,x,[instalaciones],[x]
7,7,Colombia,NaN,11 noches,julio de 2023,Persona que viaja sola,21 de julio de 2023,Excelente lugar!,10,"La limpieza, todo tal cual la foto, pero lo me...",Todo estuvo muy bien. Recomendado al 100%,Sí,la limpieza todo tal cual la foto pero lo me...,todo estuvo muy bien recomendado al,"[limpieza, tal, foto, mejor, anfitrion, super,...","[bien, recomendado]"
8,8,Colombia,NaN,1 noche,junio de 2023,En pareja,2 de julio de 2023,Acogedor,10,Apto cómodo y acogedor. Cerca a las zonas de i...,Todo perfecto.,Sí,apto cómodo y acogedor cerca a las zonas de i...,todo perfecto,"[apto, comodo, acogedor, cerca, zonas, interes...",[perfecto]
49,49,Colombia,NaN,1 noche,mayo de 2024,En pareja,7 de mayo de 2024,Excepcional,10,La ubicación y las instalaciones,Que no ofrecen desayuno u otra comida dentro d...,Sí,la ubicación y las instalaciones,que no ofrecen desayuno u otra comida dentro d...,"[ubicacion, instalaciones]","[ofrecen, desayuno, u, comida, dentro, hostal]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438164,438164,Colombia,Apartamento de 1 dormitorio,1 noche,septiembre de 2024,Persona que viaja sola,7 de septiembre de 2024,Excelente apto,10,"La comodidad del apto, lo amplio.",Quizás hubo una pequeña confusión a la hora de...,Sí,la comodidad del apto lo amplio,quizás hubo una pequeña confusión a la hora de...,"[comodidad, apto, amplio]","[quizas, pequena, confusion, hora, reserva, su..."
438166,438166,Colombia,Habitación Doble con baño compartido,1 noche,octubre de 2024,En pareja,26 de octubre de 2024,servicio y calidad,10,todo super bien gracias,todo muy bien gracias,Sí,todo super bien gracias,todo muy bien gracias,"[super, bien, gracias]","[bien, gracias]"
438176,438176,Colombia,Apartamento de 1 dormitorio,4 noches,agosto de 2024,En pareja,22 de agosto de 2024,Excepcional,10,"La anfitriona excelente muy atenta, pendiente ...",Hace falta mayor visibilizacion tanto en la no...,Sí,la anfitriona excelente muy atenta pendiente ...,hace falta mayor visibilizacion tanto en la no...,"[anfitriona, excelente, atenta, pendiente, bie...","[hace, falta, mayor, visibilizacion, nomenclat..."
438177,438177,Colombia,Apartamento Superior de 2 dormitorios,1 noche,octubre de 2024,Persona que viaja sola,30 de octubre de 2024,Genial,10,La limpieza,Todo estuvo bien,Sí,la limpieza,todo estuvo bien,[limpieza],[bien]


# Lemmatización

In [ ]:
nlp = spacy.load("es_core_news_sm")


all_lemmas_pos = []
all_lemmas_neg = []

for lista in Corpus_DF["Tokens_Cosas_Positivas_limpias"]:
    doc = nlp(' '.join(lista))

    all_lemmas_pos.append([token.lemma_ for token in doc])

for lista in Corpus_DF["Tokens_Cosas_Negativas_limpias"]:
    doc = nlp(' '.join(lista))  # Join the words in the current 'lista'
    # Append the lemmas for the current row to the all_lemmas list
    all_lemmas_neg.append([token.lemma_ for token in doc])

# Assign the all_lemmas list to the 'lemmas' column of the DataFrame
Corpus_DF["lemmas_pos"] = all_lemmas_pos
Corpus_DF["lemmas_neg"] = all_lemmas_neg

Corpus_DF

,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña,Cosas_Positivas_limpias,Cosas_Negativas_limpias,Tokens_Cosas_Positivas_limpias,Tokens_Cosas_Negativas_limpias,lemmas_pos,lemmas_neg
5,5,Colombia,NaN,1 noche,septiembre de 2023,Persona que viaja sola,5 de septiembre de 2023,Como en casa !,10,La atención y la tranquilidad tiene todo lo qu...,Nada,Sí,la atención y la tranquilidad tiene todo lo qu...,nada,"[atencion, tranquilidad, necesita, ir, laborar]",[],"[atencion, tranquilidad, necesitar, ir, laborar]",[]
6,6,Colombia,NaN,1 noche,julio de 2023,En pareja,2 de agosto de 2023,Excepcional,10,Las instalaciones,X,Sí,las instalaciones,x,[instalaciones],[x],[instalación],[x]
7,7,Colombia,NaN,11 noches,julio de 2023,Persona que viaja sola,21 de julio de 2023,Excelente lugar!,10,"La limpieza, todo tal cual la foto, pero lo me...",Todo estuvo muy bien. Recomendado al 100%,Sí,la limpieza todo tal cual la foto pero lo me...,todo estuvo muy bien recomendado al,"[limpieza, tal, foto, mejor, anfitrion, super,...","[bien, recomendado]","[limpiecer, tal, foto, mejor, anfitrion, super...","[bien, recomendado]"
8,8,Colombia,NaN,1 noche,junio de 2023,En pareja,2 de julio de 2023,Acogedor,10,Apto cómodo y acogedor. Cerca a las zonas de i...,Todo perfecto.,Sí,apto cómodo y acogedor cerca a las zonas de i...,todo perfecto,"[apto, comodo, acogedor, cerca, zonas, interes...",[perfecto],"[apto, comodo, acogedor, cerca, zona, inter, c...",[perfecto]
49,49,Colombia,NaN,1 noche,mayo de 2024,En pareja,7 de mayo de 2024,Excepcional,10,La ubicación y las instalaciones,Que no ofrecen desayuno u otra comida dentro d...,Sí,la ubicación y las instalaciones,que no ofrecen desayuno u otra comida dentro d...,"[ubicacion, instalaciones]","[ofrecen, desayuno, u, comida, dentro, hostal]","[ubicacion, instalaciones]","[ofrecer, desayuno, u, comida, dentro, hostal]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438164,438164,Colombia,Apartamento de 1 dormitorio,1 noche,septiembre de 2024,Persona que viaja sola,7 de septiembre de 2024,Excelente apto,10,"La comodidad del apto, lo amplio.",Quizás hubo una pequeña confusión a la hora de...,Sí,la comodidad del apto lo amplio,quizás hubo una pequeña confusión a la hora de...,"[comodidad, apto, amplio]","[quizas, pequena, confusion, hora, reserva, su...","[comodidad, apto, amplio]","[quiza, pequén, confusion, hora, reservar, sub..."
438166,438166,Colombia,Habitación Doble con baño compartido,1 noche,octubre de 2024,En pareja,26 de octubre de 2024,servicio y calidad,10,todo super bien gracias,todo muy bien gracias,Sí,todo super bien gracias,todo muy bien gracias,"[super, bien, gracias]","[bien, gracias]","[super, bien, gracias]","[bien, gracias]"
438176,438176,Colombia,Apartamento de 1 dormitorio,4 noches,agosto de 2024,En pareja,22 de agosto de 2024,Excepcional,10,"La anfitriona excelente muy atenta, pendiente ...",Hace falta mayor visibilizacion tanto en la no...,Sí,la anfitriona excelente muy atenta pendiente ...,hace falta mayor visibilizacion tanto en la no...,"[anfitriona, excelente, atenta, pendiente, bie...","[hace, falta, mayor, visibilizacion, nomenclat...","[anfitrionar, excelente, atento, pendiente, bi...","[hacer, falta, mayor, visibilizacion, nomencla..."
438177,438177,Colombia,Apartamento Superior de 2 dormitorios,1 noche,octubre de 2024,Persona que viaja sola,30 de octubre de 2024,Genial,10,La limpieza,Todo estuvo bien,Sí,la limpieza,todo estuvo bien,[limpieza],[bien],[limpiecer],[bien]


# Vectorización con CountVercorizer

In [ ]:
#Convertir las listas de tokens en una lista de cadenas (una cadena por lista de tokens)
Corpus_DF['lemmas_pos_str'] = Corpus_DF['Tokens_Cosas_Positivas_limpias'].apply(lambda x: ' '.join(x))
Corpus_DF['lemmas_neg_str'] = Corpus_DF['Tokens_Cosas_Negativas_limpias'].apply(lambda x: ' '.join(x))

# Crear un CountVectorizer para transformar las palabras en vectores
vectorizer = CountVectorizer()

#Ajustar el vectorizador con ambos conjuntos de datos
all_text = Corpus_DF['lemmas_pos_str'].tolist() + Corpus_DF['lemmas_neg_str'].tolist()
vectorizer.fit(all_text)

# Transformar ambos conjuntos de datos usando el vectorizador ajustado
X = vectorizer.transform(Corpus_DF['lemmas_pos_str'])
A = vectorizer.transform(Corpus_DF['lemmas_neg_str'])

# Convertir la matriz de vectores a un DataFrame
vectorized_df_pos = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
vectorized_df_neg = pd.DataFrame(A.toarray(), columns=vectorizer.get_feature_names_out())
vectorized_df_pos

,abajo,abandonadas,abandonar,abandone,abandoned,abanico,abastecimiento,abasto,abertura,abholung,...,zoe,zona,zonas,zone,zones,zuchinni,zula,zuleima,zully,zulma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20423,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20424,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
vectorized_df_neg

,abajo,abandonadas,abandonar,abandone,abandoned,abanico,abastecimiento,abasto,abertura,abholung,...,zoe,zona,zonas,zone,zones,zuchinni,zula,zuleima,zully,zulma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20423,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20424,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Vectorización con TF-IDF

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,3))

# Combine all text data for a unified vocabulary
all_text = Corpus_DF['Tokens_Cosas_Positivas_limpias'].apply(lambda x: ' '.join(x)).tolist() + Corpus_DF['Tokens_Cosas_Negativas_limpias'].apply(lambda x: ' '.join(x)).tolist()

# Fit the TfidfVectorizer on the combined data
tfidf.fit(all_text)

# Transform positive and negative tokens using the fitted vectorizer
tfidf_matrix_pos = tfidf.transform(Corpus_DF['Tokens_Cosas_Positivas_limpias'].apply(lambda x: ' '.join(x)))
tfidf_matrix_neg = tfidf.transform(Corpus_DF['Tokens_Cosas_Negativas_limpias'].apply(lambda x: ' '.join(x)))

# Get feature names (words)
words = tfidf.get_feature_names_out()

# Create DataFrames
tfidf_df_pos = pd.DataFrame(tfidf_matrix_pos.toarray(), columns=words)
tfidf_df_neg = pd.DataFrame(tfidf_matrix_neg.toarray(), columns=words)

tfidf_df_pos

,abajo,abajo comedor,abajo dijeron,abajo dijeron prestaban,abajo elegir,abajo elegir restaurantes,abajo escucha,abajo escucha resto,abajo hotel,abajo hotel plazoleta,...,zuleima realmente,zuleima realmente buena,zully,zully attention,zully attention welcoming,zully jose,zully jose incluidos,zulma,zulma chef,zulma chef pedro
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tfidf_df_neg

,abajo,abajo comedor,abajo dijeron,abajo dijeron prestaban,abajo elegir,abajo elegir restaurantes,abajo escucha,abajo escucha resto,abajo hotel,abajo hotel plazoleta,...,zuleima realmente,zuleima realmente buena,zully,zully attention,zully attention welcoming,zully jose,zully jose incluidos,zulma,zulma chef,zulma chef pedro
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
